## Calculate Euclidean distance to nearest charger 
In this notebook we compute the distance between each candidate site (Exit dataset from MJBA) and the nearest existing DCFC charger (from NREL). We do this by using the Shapely's `nearest_point()` function, which identifies the point in a multipoint geometry that is nearest a provided point, and then use the `distance()` function to compute the Euclidean distance between the two. 

This approach uses the procesees outlined here:<br>
https://automating-gis-processes.github.io/2017/lessons/L3/nearest-neighbour.html

In [ ]:
#Import packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

In [ ]:
#Import exit features
exits_gdf = gpd.read_file('../Data/exits_flood.shp')
exits_gdf.head()

#### Converting a CSV file (with coordinates) to a geodataframe
The NREL data are stored as a CSV file. We need to get this into a geodataframe to facilitate analysis.

* To do this, we first import the CSV into a Pandas dataframe as this has all the data we need to construct the geodataframe. 

In [ ]:
#Import DCFC csv as a Pandas dataframe, importing only the longitude and latitude columns
dcfc_df = pd.read_csv('../Data/NREL/DCFC.csv',usecols=['longitude','latitude'])
dcfc_df.head()

* Next, we create a list of shapely "Point" objects from lists of longitude and latitude pairs, constructed by "zip"ing the two lists together.

In [ ]:
#Construct points from long/lat pairs
geom = [Point(xy) for xy in zip(dcfc_df['longitude'],dcfc_df['latitude'])]
#Show the first 5 records
geom[:5]

* And finally, we create a Geopandas geodataframe by specifying the Pandas dataframe, the list of geometries, and a coordinate reference system to use. (These are lat/long data so we use WGS84, i.e. epsg #4326.)

In [ ]:
#Convert to a spatial dataframe
dcfc_gdf = gpd.GeoDataFrame(dcfc_df,geometry=geom,crs=4326)
#Show the results
dcfc_gdf.plot();

#### Coordinate reference systems
Because we are going to measure distances, we need to convert our two geodataframes, which are using geographic coordinate reference systems, to a projected crs. We'll use UTM Zone 17N (epsg=`32617`), employing the `to_crs()` function to tranform our datsets.

In [ ]:
#Reveal crfs of each
print(exits_gdf.crs)
print(dcfc_gdf.crs)

In [ ]:
#Transform to a projected crf
exits_gdf = exits_gdf.to_crs(32617)
dcfc_gdf = dcfc_gdf.to_crs(32617)

In [ ]:
#Reveal crfs of each: they should be different
print(exits_gdf.crs)
print(dcfc_gdf.crs)

#### Nearest point analysis
* To compute the nearest DCFC to each exit, we must first collapse our individual DCFC point locations into a single *mulitpoint* geometry object. 

In [ ]:
#Collapse the dcfc points into one multipoint object
mp_dcfc = dcfc_gdf['geometry'].unary_union

* Before tackling all Exit points, let's examine the workflow for a single point.

In [ ]:
#Get the first exit point
thePoint = exits_gdf.at[0,'geometry']

* The `nearest_points()` function returns a pair of points, with the first point being the source point, and the second being the nearest point in the supplied multipoint geometry.<br><br>
In the statement below, we save the two outputs into separate objects

In [ ]:
#Apply the nearest_points function
from_pt, to_pt = nearest_points(thePoint,mp_dcfc)

* Now we can easily compute the distance between the two points using the `distance()` function...

In [ ]:
#Compute the distance between the points
distance = from_pt.distance(to_pt)
distance

→ The first exit is 165800 m from the nearest charger (straight line distance). 

Now let's put this into production mode and compute nearest distances from all points. We'll do this by constructing a Python function and then applying that function to all points. 

In [ ]:
#Put it all together in a function
def distance_to_dcfc(from_point,to_points):
    distance=from_point.distance(nearest_points(from_point,to_points)[1])
    #Convert to miles
    return int(distance/1609.34)

In [ ]:
#Test it
distance_to_dcfc(thePoint,mp_dcfc)

In [ ]:
#Apply it to all exit features
exits_gdf['Dist_to_DCFC']=exits_gdf['geometry'].apply(lambda x: distance_to_dcfc(x,mp_dcfc))
exits_gdf.head()

In [ ]:
#Save the output to a csv file
exits_gdf.to_file('../Data/Exits_distance_to_DCFC.shp')